In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
#Loading the data
today = str(date.today())
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")

In [4]:
#Profiling the data
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


In [5]:
df_customer_transaction.head(10)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ORTIZ,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,NIENOW,LEA,2000-11-26
2,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
3,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
4,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
5,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
6,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
7,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
8,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10
9,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10


In [6]:
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(130653, 4)
(130653, 5)


In [7]:
print(df_branch_service['txn_id'].nunique())
print(df_customer_transaction['txn_id'].nunique())

62354
62354


In [8]:
#Dropping duplicates
df_branch_service = df_branch_service.drop_duplicates(subset=['txn_id'])
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=['txn_id'])

In [9]:
#confirming shape
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(62354, 4)
(62354, 5)


In [10]:
#checking null values count per column
df_branch_service.isnull().sum()

txn_id             0
branch_name     8880
service            0
price          11082
dtype: int64

In [11]:
df_customer_transaction.isnull().sum()

txn_id        0
avail_date    0
last_name     0
first_name    0
birthday      0
dtype: int64

In [12]:
#fill up null values with forward fill for branch_name
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', '', None,
       'MayMall', 'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [13]:
#'' and None are empty values, fill them with forward fill
# first, let '' be null
df_branch_service['branch_name'] = df_branch_service.replace('',np.nan).groupby('txn_id')['branch_name'].transform('first')
# then fill null, use forward and backward to fill
df_branch_service['branch_name'] = df_branch_service['branch_name'].ffill().bfill()

In [14]:
#confirming branch_name has no null
df_branch_service.isnull().sum()

txn_id             0
branch_name        0
service            0
price          11082
dtype: int64

In [15]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', 'MayMall',
       'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [16]:
# group mean fill prices
df_branch_service['price'] = df_branch_service['price'].fillna(df_branch_service.groupby(['branch_name','service'])['price'].transform('mean'))

In [17]:
#confirming price has no null
df_branch_service.isnull().sum()

txn_id         0
branch_name    0
service        0
price          0
dtype: int64

In [18]:
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,42.651993
1,TXN-14642,Starmall,HairColor,69.583267
2,TXN-60295,SmallMall,FootSpa,80.037329
6,TXN-40462,Starmall,HairColor,69.583267
8,TXN-08102,RobinsonsMall,HairColor,66.735226
12,TXN-64262,SmallMall,Rebond,312.646002
15,TXN-41934,RobinsonsMall,HairColor,66.735226
19,TXN-39050,Starmall,Manicure,43.364281
21,TXN-43150,Starmall,Haircut,51.769018
22,TXN-49423,RobinsonsMall,Rebond,302.828204


In [19]:
df_merged = pd.merge(df_customer_transaction, df_branch_service)

In [20]:
#profiling of merged dataframe
df_merged.isnull().sum()

txn_id         0
avail_date     0
last_name      0
first_name     0
birthday       0
branch_name    0
service        0
price          0
dtype: int64

In [21]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
21516,TXN-49458,2019-06-19,greenfelder,hilma,2003-02-27,SmallMall,Manicure,0.000000
39452,TXN-51287,2029-06-18,"Waters,,,,,,,,",Abigayle........,1999-12-27,Starmall,Haircut,66.123457
27792,TXN-18347,2013-09-08,Hahn,Erling,2000-02-23,MallOfAsia,Pedicure,77.987989
41786,TXN-33073,2008-04-21,"Muller,,,,,,,,",Ashly........,2003-10-14,MayMall,FootSpa,100.121230
21769,TXN-13500,2021-06-20,shanahan,jeanette,2006-10-17,FrankMall,Manicure,0.000000
890,TXN-28818,2006-05-12,GOTTLIEB,IDELLA,2009-11-02,MayMall,Pedicure,60.673224
21197,TXN-47677,2022-07-12,raynor,lawson,2001-09-01,Starmall,Rebond,0.000000
1480,TXN-35961,2024-04-11,BOYER,ELYSSA,2000-03-12,MallOfAsia,Rebond,316.049754
41779,TXN-35151,2022-05-25,"Mosciski,,,,,,,,",Chaim........,1997-04-22,FrankMall,Pedicure,77.987989
23370,TXN-36975,2018-11-12,Carroll,Ophelia,2006-06-21,Starmall,Pedicure,77.987989


In [22]:
#filter alphabet only in last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\juanc\AppData\Local\Temp\ipykernel_19296\1982014538.py:2: SyntaxWarning: invalid escape sequence '\W'
  df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_19296\1982014538.py:3: SyntaxWarning: invalid escape sequence '\W'
  df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)


In [23]:
#uppercase for last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.upper()
df_merged['first_name'] = df_merged['first_name'].str.upper()

In [24]:
#check if no special characters and all uppercase for names
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
45774,TXN-17644,2007-11-29,LINDGREN,MEAGHAN,1990-02-28,Starmall,Rebond,400.231230
41753,TXN-21426,2007-01-19,BREKKE,DALLAS,2000-12-03,RobinsonsMall,FootSpa,100.121230
47556,TXN-46595,2006-04-15,REMPEL,AUSTEN,2008-12-26,Starmall,FootSpa,100.121230
40410,TXN-45268,2030-12-04,ROLFSON,TROY,1997-03-04,MallOfAsia,Pedicure,77.987989
50431,TXN-01719,2018-02-07,MAYER,HIRAM,2007-02-23,RobinsonsMall,Manicure,55.232400
56825,TXN-58571,2027-08-19,LEMKE,TEAGAN,2004-04-05,FrankMall,Haircut,66.123457
28416,TXN-23501,2008-10-14,CASPER,PRUDENCE,1994-07-15,Megamall,Haircut,66.123457
8271,TXN-04022,2018-03-09,GULGOWSKI,SALVATORE,1991-11-07,FrankMall,HairColor,67.045741
19391,TXN-26500,2025-07-25,BEDNAR,SIMEON,2007-08-27,RobinsonsMall,FootSpa,0.000000
62174,TXN-48186,2022-11-06,BALISTRERI,DEVEN,2002-05-03,SmallMall,FootSpa,100.121230


In [25]:
#profiling date columns
print(df_merged['birthday'].min())
print(df_merged['birthday'].max())

print(df_merged['avail_date'].min())
print(df_merged['avail_date'].max())

print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

1990-01-01
2010-12-30
2005-01-01
2030-12-30
<bound method NDFrame.describe of 0        2030-09-08
1        2026-05-26
2        2006-09-25
3        2021-08-21
4        2010-04-03
            ...    
62349    2022-11-21
62350    2020-04-14
62351    2030-01-26
62352    2021-02-27
62353    2025-01-05
Name: avail_date, Length: 62354, dtype: object>
<bound method NDFrame.describe of 0        1990-07-08
1        2000-11-26
2        1993-05-22
3        2002-11-25
4        2003-07-10
            ...    
62349    1997-03-06
62350    1996-05-28
62351    1996-05-28
62352    1995-03-29
62353    1993-01-10
Name: birthday, Length: 62354, dtype: object>
          txn_id  avail_date  last_name first_name    birthday    branch_name  \
9      TXN-49423  2008-08-13      BERGE    RASHEED  2010-02-05  RobinsonsMall   
17     TXN-35244  2005-11-23     HAMMES     EUNICE  2006-06-13      SmallMall   
97     TXN-23898  2005-05-28   BOTSFORD     GIANNI  2009-03-30  RobinsonsMall   
101    TXN-17974  2008-04-12  

In [26]:
#data type of birthday and avail_date should be datetime instead of object
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')

#confirming
print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

<bound method NDFrame.describe of 0       2030-09-08
1       2026-05-26
2       2006-09-25
3       2021-08-21
4       2010-04-03
           ...    
62349   2022-11-21
62350   2020-04-14
62351   2030-01-26
62352   2021-02-27
62353   2025-01-05
Name: avail_date, Length: 62354, dtype: datetime64[ns]>
<bound method NDFrame.describe of 0       1990-07-08
1       2000-11-26
2       1993-05-22
3       2002-11-25
4       2003-07-10
           ...    
62349   1997-03-06
62350   1996-05-28
62351   1996-05-28
62352   1995-03-29
62353   1993-01-10
Name: birthday, Length: 62354, dtype: datetime64[ns]>


In [27]:
#removing rows with later date than current date
df_merged = df_merged[(df_merged['avail_date'] <= today) & (df_merged['birthday'] <= today)]

In [28]:
#removing rows when avail_date happens before birthday
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

In [29]:
#confirming
print(df_merged['birthday'].max())
print(df_merged['avail_date'].max())

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

2010-12-30 00:00:00
2023-11-22 00:00:00
Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday, branch_name, service, price]
Index: []


In [30]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
31201,TXN-29793,2007-07-20,KLING,VAUGHN,2003-07-15,MallOfAsia,Pedicure,77.987989
3596,TXN-47512,2019-11-23,DICKINSON,STEPHANY,1998-08-12,RobinsonsMall,HairColor,66.735226
54874,TXN-37049,2016-01-02,RIPPIN,RUTHE,2004-09-10,SmallMall,HairColor,88.093930
22792,TXN-24319,2018-07-13,MCKENZIE,ROWENA,1991-12-29,SmallMall,FootSpa,100.121230
10654,TXN-38888,2022-05-11,THIEL,MARIELA,1999-06-19,Megamall,Manicure,41.759774
44151,TXN-13124,2010-01-19,RUECKER,AALIYAH,1990-01-24,RobinsonsMall,NailColor,30.123790
54194,TXN-10069,2005-10-25,ULLRICH,CANDICE,2001-12-30,Megamall,Rebond,400.231230
22872,TXN-58083,2023-08-17,SCHIMMEL,ORA,2002-06-25,MayMall,HairColor,88.093930
4380,TXN-39313,2021-11-20,MCCLURE,AVERY,2005-12-25,RobinsonsMall,Haircut,51.196577
15447,TXN-05263,2017-02-26,SCHMITT,BRIONNA,2002-08-03,SmallMall,Pedicure,0.000000


In [31]:
df_merged.shape

(43131, 8)

In [32]:
import sqlite3

In [33]:
db_connect = sqlite3.connect('datadb')

In [34]:
cur = db_connect.cursor()

In [35]:
cur.execute("create table if not exists datalab (txn_id varchar(45) not null primary key,avail_data date, last_name varchar(20),birthday date,branch_name varchar(30), service varcahr(30), price double)")
db_connect.commit()

In [36]:
cur.execute ("SELECT name from sqlite_master where type='table' order by name")
results = cur.fetchall()
print(results)

[('datalab',)]


In [37]:
df_merged.to_sql('datalab', db_connect, if_exists='replace', index=False)

43131

In [38]:
latest_df_merged=pd.read_sql_query('select * from datalab limit 10', db_connect)

In [39]:
latest_df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25 00:00:00,LESCH,FLETA,1993-05-22 00:00:00,SmallMall,FootSpa,80.037329
1,TXN-40462,2021-08-21 00:00:00,KUHN,TOD,2002-11-25 00:00:00,Starmall,HairColor,69.583267
2,TXN-08102,2010-04-03 00:00:00,JOHNSON,MILTON,2003-07-10 00:00:00,RobinsonsMall,HairColor,66.735226
3,TXN-64262,2018-11-23 00:00:00,HACKETT,MACI,2005-06-20 00:00:00,SmallMall,Rebond,312.646002
4,TXN-59988,2021-02-01 00:00:00,BASHIRIAN,BRICE,2004-07-20 00:00:00,RobinsonsMall,Rebond,302.828204
5,TXN-08158,2023-08-05 00:00:00,BATZ,TIMMOTHY,2008-10-06 00:00:00,MayMall,NailColor,22.716300
6,TXN-37609,2022-09-04 00:00:00,KUNDE,MARIANA,1996-10-29 00:00:00,MallOfAsia,FootSpa,78.040113
7,TXN-31294,2018-01-04 00:00:00,BREITENBERG,MADGE,2006-06-22 00:00:00,RobinsonsMall,Haircut,51.196577
8,TXN-21732,2017-03-16 00:00:00,TURCOTTE,HELLEN,1996-03-25 00:00:00,RobinsonsMall,FootSpa,75.620543
9,TXN-31732,2013-04-28 00:00:00,LOCKMAN,CHADD,2003-06-17 00:00:00,RobinsonsMall,Manicure,41.900441


In [40]:
latest_df_merged=pd.read_sql_query('select * from datalab', db_connect)

In [41]:
latest_df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25 00:00:00,LESCH,FLETA,1993-05-22 00:00:00,SmallMall,FootSpa,80.037329
1,TXN-40462,2021-08-21 00:00:00,KUHN,TOD,2002-11-25 00:00:00,Starmall,HairColor,69.583267
2,TXN-08102,2010-04-03 00:00:00,JOHNSON,MILTON,2003-07-10 00:00:00,RobinsonsMall,HairColor,66.735226
3,TXN-64262,2018-11-23 00:00:00,HACKETT,MACI,2005-06-20 00:00:00,SmallMall,Rebond,312.646002
4,TXN-59988,2021-02-01 00:00:00,BASHIRIAN,BRICE,2004-07-20 00:00:00,RobinsonsMall,Rebond,302.828204
...,...,...,...,...,...,...,...,...
43126,TXN-42627,2009-03-28 00:00:00,GREENHOLT,MARCELLUS,1999-10-15 00:00:00,RobinsonsMall,FootSpa,100.121230
43127,TXN-17127,2012-08-19 00:00:00,CONROY,ABELARDO,1991-01-17 00:00:00,Megamall,Haircut,66.123457
43128,TXN-28428,2022-11-21 00:00:00,MULLER,LIAM,1997-03-06 00:00:00,Megamall,FootSpa,100.121230
43129,TXN-55630,2020-04-14 00:00:00,PURDY,HARMON,1996-05-28 00:00:00,FrankMall,HairColor,88.093930
